# 256×256のnpyファイルを用いたSwin-Smallモデル学習ノートブック
このノートブックでは、256×256のnpy画像データを用いてSwin Transformer (swin_s) モデルによる脳動脈瘤検出タスクの学習を行います。

- データ: `series_npy/256/` 配下のnpyファイル
- ラベル: `train.csv`
- モデル: Swin-Small (timm)

---

## 1. 必要なライブラリのインポート

In [1]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import timm
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import random


c:\Users\emanon\anaconda3\envs\Kaggle\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 2. データセットの準備
- train.csvの読み込み
- image_256_path_df.csvから画像パスを取得
- ラベルと画像パスの紐付け

In [3]:
# パス設定
TRAIN_CSV = '../train.csv'
IMG_PATH_DF = '../npy_path/image_256_path_df.csv'
NPY_ROOT = '../series_npy/256/'

# train.csvの読み込み
train_df = pd.read_csv(TRAIN_CSV)
# image_256_path_df.csvの読み込み
img_path_df = pd.read_csv(IMG_PATH_DF)

# SeriesInstanceUIDでマージ
merged_df = pd.merge(train_df, img_path_df, on='SeriesInstanceUID', how='inner')

# 画像パスとラベルの確認
print(merged_df[['SeriesInstanceUID', 'image_path', 'Aneurysm Present']].head())


KeyError: 'SeriesInstanceUID'

## 3. データ前処理とデータローダの作成
- npyファイルを読み込むDatasetクラスの実装
- データ拡張・正規化
- train/val分割とDataLoader作成

In [ ]:
from torchvision import transforms

class NPYDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        npy_path = row['image_path']
        img = np.load(npy_path).astype(np.float32)  # (H, W) or (C, H, W)
        if img.ndim == 2:
            img = np.expand_dims(img, axis=0)  # (1, H, W)
        if self.transform:
            img = self.transform(torch.from_numpy(img))
        label = row['Aneurysm Present']
        return img, torch.tensor(label, dtype=torch.float32)

# データ分割
train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42, stratify=merged_df['Aneurysm Present'])

# 前処理・データ拡張
transform = transforms.Compose([
    transforms.Lambda(lambda x: x/255.0),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = NPYDataset(train_df, transform=transform)
val_dataset = NPYDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)


## 4. Swin Transformer (swin_s) モデルの構築
- timmからswin_sをロードし、出力層を本タスク用に変更

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 入力チャンネル数（1チャンネル画像の場合）
IN_CHANS = 1
NUM_CLASSES = 1  # バイナリ分類

model = timm.create_model('swin_small_patch4_window7_224', pretrained=True, in_chans=IN_CHANS)
# 出力層をタスクに合わせて変更
model.head = nn.Linear(model.head.in_features, NUM_CLASSES)
model = model.to(device)

print(model)


## 5. 損失関数・最適化手法の設定
- BCEWithLogitsLoss, AdamW

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)


## 6. 学習ループの実装
- train/valのループ
- 損失・評価指標の記録

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        outputs = outputs.squeeze(-1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)

def eval_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    preds, gts = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            outputs = outputs.squeeze(-1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            preds.extend(torch.sigmoid(outputs).cpu().numpy())
            gts.extend(labels.cpu().numpy())
    return running_loss / len(loader.dataset), np.array(preds), np.array(gts)

EPOCHS = 10
best_auc = 0.0
train_losses, val_losses, val_aucs = [], [], []
best_model_wts = None

for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_preds, val_gts = eval_one_epoch(model, val_loader, criterion, device)
    val_auc = roc_auc_score(val_gts, val_preds)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_aucs.append(val_auc)
    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val AUC: {val_auc:.4f}")
    if val_auc > best_auc:
        best_auc = val_auc
        best_model_wts = model.state_dict().copy()


## 7. 検証・評価指標の計算
- バリデーションAUC・学習曲線の可視化

In [ ]:
# 学習曲線の可視化
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.legend()
plt.title('Loss')
plt.subplot(1,2,2)
plt.plot(val_aucs, label='Val AUC')
plt.legend()
plt.title('Validation AUC')
plt.show()

print(f'Best Validation AUC: {best_auc:.4f}')


## 8. 学習済みモデルの保存
- 最良モデルの重みを保存

In [ ]:
# 最良モデルの保存
os.makedirs('model_weights', exist_ok=True)
torch.save(best_model_wts, 'model_weights/swin_s_best_256.pth')
print('Saved best model weights.')
